In [1]:
!mkdir dataset

In [2]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 99% 80.0M/80.9M [00:01<00:00, 94.6MB/s]
100% 80.9M/80.9M [00:01<00:00, 73.1MB/s]


In [3]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('the dataset is extracted')

the dataset is extracted


In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [8]:
twitter_data.shape

(1599999, 6)

In [9]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
column_names=['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding='ISO-8859-1')

In [11]:
twitter_data.shape

(1600000, 6)

In [12]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [14]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [15]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [16]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [18]:
#0=negative tweet
#1=positive tweet
stemmer = PorterStemmer()
def stemming(tweet):
    tweet = re.sub('[^a-zA-Z]', ' ', tweet)
    tweet = tweet.lower()
    tweet = tweet.split()
    tweet = [stemmer.stem(word) for word in tweet if word not in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    return tweet


In [21]:
pos_data = twitter_data[twitter_data['target'] == 1].iloc[:100000]
neg_data = twitter_data[twitter_data['target'] == 0].iloc[:100000]
twitter_data = pd.concat([pos_data, neg_data], axis=0)

In [22]:
twitter_data['text'] = twitter_data['text'].apply(stemming)

In [23]:
twitter_data.head()

,target,id,date,flag,user,text
800000,1,1467822272,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,ersle,love health uandpet u guy r best
800001,1,1467822273,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,becca210,im meet one besti tonight cant wait girl talk
800002,1,1467822283,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,Wingman29,darealsunisakim thank twitter add sunisa got m...
800003,1,1467822287,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,katarinka,sick realli cheap hurt much eat real food plu ...
800004,1,1467822293,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,_EmilyYoung,lovesbrooklyn effect everyon


In [25]:
print(twitter_data['text'])

800000                     love health uandpet u guy r best
800001        im meet one besti tonight cant wait girl talk
800002    darealsunisakim thank twitter add sunisa got m...
800003    sick realli cheap hurt much eat real food plu ...
800004                         lovesbrooklyn effect everyon
                                ...                        
99995           son develop new habit wake second pot coffe
99996                     look like router broke tweet fone
99997              realli dont want colleg right wish sunni
99998                                    flossa offer pepto
99999                          josiehobo would sooooo revis
Name: text, Length: 200000, dtype: object


In [26]:
print(twitter_data['target'])

800000    1
800001    1
800002    1
800003    1
800004    1
         ..
99995     0
99996     0
99997     0
99998     0
99999     0
Name: target, Length: 200000, dtype: int64


In [27]:
x=twitter_data['text'].values
y=twitter_data['target'].values

In [30]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,stratify=y, random_state=2)

In [31]:
print(x.shape, x_train.shape, x_test.shape)

(200000,) (160000,) (40000,)


In [32]:
print(x_train)

['hate twitter type limit'
 'simonforfilm thumb lovefilmluci take time repli blog post lovefilm http bit ly xz hm good servic'
 'ap ap ap earth movi ap ap' ...
 'didilaksana almost went garden thought road lobbi'
 'wish sunday want internet muh phone'
 'achristopherf unfortun famili commit whole day long dress get readi']


In [50]:
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)


In [58]:
model = LogisticRegression()


In [60]:
model.fit(x_train_tfidf, y_train)

LogisticRegression()

In [66]:
x_pred = model.predict(x_train_tfidf)
accuracy = accuracy_score(y_train, x_pred)



In [67]:
print('Accuracy score of the training data:', accuracy)

Accuracy score of the training data: 0.8201125


In [68]:
y_pred = model.predict(x_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)



In [69]:
print('Accuracy score of the testing data:', accuracy)

Accuracy score of the testing data: 0.76965


model accuracy =76.96%

In [70]:
import pickle

In [71]:
filename ='trained.sav'
pickle.dump(model, open(filename, 'wb'))

In [73]:
loaded_model=pickle.load(open('/content/trained.sav', 'rb'))

In [74]:
print(x_test[300])

iamdiddi hi ya je di realli diddi brand new di da twitter crazi jst hittin irish shore love ur music ur legend xx


In [80]:
x_new=x_test[300]
x_new=vectorizer.transform([x_new])

In [81]:
prediction=model.predict(x_new)
print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

[1]
Positive Tweet
